In [7]:
import pickle
from qiskit import BasicAer, execute
from qiskit.quantum_info.states.measures import state_fidelity

def simulate_circ(circ, simulator='statevector_simulator'):
    backend = BasicAer.get_backend(simulator)
    job = execute(circ, backend=backend)
    result = job.result()
    outputstate = result.get_statevector(circ)
    return outputstate

dirname = './data'
full_circ = pickle.load(open( '%s/full_circ.p'%dirname, 'rb' ))
original_sv = simulate_circ(full_circ)
reconstructed_sv = pickle.load(open( '%s/reconstructed.p'%dirname, 'rb' ))

print(state_fidelity(original_sv,reconstructed_sv))

1.5803186408950502e-05
